# Gravity

## Tips

Learn `Python` by going through the suggested online tutorials and the `PythonMinimum` notebook.

  * Python tutorials:
     * [A short introduction](https://realpython.com/python-first-steps/)
     * [A more complete introduction](https://www.w3schools.com/python/default.asp)
     * PythonMinimum
  * Use __esc r__ to disable a cell
  * Use __esc y__ to reactivate it
  * Use __esc m__ to go to markdown mode. **Markdown** is the typesetting language used in jupyter notebooks.
  * In a markdown cell, double tap the mouse or glide pad (on your laptop) to go to edit mode. 
  * Shift + return to execute a cell (including markdown cells).
  * If the equations don't typeset, try double tapping the cell again, and re-execute it.


## Goal

This notebook provides a gentle introduction to gravitational interactions.

## Newton's Law of Gravity

According to Newton, two objects of 1 and 2 or mass $m_1$ and $m_2$ at positions $\vec{r}_1$ and $\vec{r}_2$, respectively, experience a gravitational pull on each other of magnitude

\begin{align}
    G \frac{m_1 m_2}{|\vec{r}_1 - \vec{r}_2|^2}
\end{align}

where $G$ is Newton's gravitational constant. But force is a vector, so let's write the gravitational force of object 2 on object 1. Define the vector that starts at object 2 and ends on
object 1, $\vec{r} = \vec{r}_1 - \vec{r}_2$ as shown in the figure. 
<img src="fig07_2objects.png" align="left" alt="2-body system" width="300"/>
<br clear="right"/>
Also define the associated unit vector $\hat{r}$. Then the gravitational force of object 2 **on** object 1 can be written as

\begin{align}
    \vec{F}_g & = - \left( G \frac{m_1 m_2}{|\vec{r}|^2} \right) \hat{r},\\
    & = - \left( G \frac{m_1 m_2}{|\vec{r}|^2} \right) \frac{\vec{r}}{|\vec{r}|},\\
    & = - G \frac{m_1 m_2}{|\vec{r}|^3} \vec{r} .
\end{align}

Why is there a negative sign?

Newton did not understand how it was possible for one object to exert a gravitational force on another that is not in contact with it. An explanation was provided by Michael Faraday in the 1840s. Michael Faraday suggested that an object, for example the Sun, exerts a force on the Earth indirectly. The Sun creates a **gravitational field** around it that extends throughout space and it is the field at the location of the Earth that exerts the gravitational force on the Earth. Likewise the gravitational field of the Earth extends all the way to the Sun and the field at the Sun's location exerts a force on the Sun. Your gravitational field also extends all the way to the Sun and exerts a force on the Sun!

We can make this idea quantitative by noticing that we can write the force of object 2 on object 1 as follows

\begin{align}
    \vec{F}_g & = m_1 \vec{g}_1 ,
\end{align}

just as we did for the projectile motion problem. By comparing the above with the expression for the gravitational force, we see that the **gravitational field** $\vec{g}_1$ at the location of object 1 can be written as 

\begin{align}
    \vec{g}_1 & = - G \frac{m_2}{|\vec{r}|^3} \vec{r} .
\end{align}

If we want the gravitational force on object 1 due to objects 2, 3, $\cdots n$, we first add their gravitational fields 

\begin{align}
    \vec{g}_1 & = -G \sum_{j=2}^n \frac{m_j}{|\vec{r}_1 - \vec{r}_j|^3} (\vec{r}_1 - \vec{r}_j) ,
\end{align}

then multiply the total gravitational field $\vec{g}_1$ by the mass $m_1$. This must be done for each of the objects in turn. Therefore, for object $i$ we need to compute all the possible vector differences $\vec{r}_i - \vec{r}_j$ with $i \neq j$. 

In this exercise, and in the associated project, we'll assume that the orbital planes of the inner planets lie in the same plane. The initial positions of the planets will be taken to be their positions on 22 September 2024 as given in the table below. (The data were provided at this [website](https://eco.mtk.nao.ac.jp/cgi-bin/koyomi/cande/planet_ecliptic_en.cgi).)


| **planet** | **longitude** (deg) | **radius**  (au)| **speed** (deg/day) |
| :--        | :--           | :--             | :--                 |
| Mercury|152.26499|0.3513275|4.86195 |
| Venus|251.88011|0.7257176|1.59162|
| Earth|000.00000|1.0035870|0.98562|
| Mars|58.834830|1.4917856|0.54435|
| Jupiter|69.554680|5.0509857|0.08809|
| Saturn|346.51910|9.6606022|0.03265|
| Uranus|54.60613|19.5701169|0.01133|
| Neptune|358.53431|29.896918|0.00610|


Positions one day later (23 September 2024).

| **planet** | **longitude** (deg) | **radius**  (au)| **speed** (deg/day) |
| :--        | :--           | :--             | :--                 |
| Mercury|157.05017|0.3569830|4.70912|
| Venus|253.47147|0.7258350|1.59110|
| Earth|000.00000|1.0033129|0.98562|
| Mars|59.37871|1.4930812|0.54341|
| Jupiter|69.64277|5.0512854|0.08808|
| Saturn|346.55175|9.6603041|0.03265|
| Uranus|54.61746|19.5699504|0.01133|
| Neptune|358.54041|29.896892|0.00610|


The data above is relative to the **heliocentric coordinate system**
<img src="fig04_heliocentric.png" align="right" alt="heliocentric coordinate system" width="400"/>
The center of this coordinate system is near the center of the Sun. In fact, the origin of this coordinate system is (by [international agreement](https://en.wikipedia.org/wiki/International_Celestial_Reference_System_and_its_realizations)) at the center of mass of the Solar System.
<br clear="right"/>



## Numerical Solution of Newton's Second Law

Recall that Newton's second law of motion for a particle of mass $m$

$$\vec{F} = m \vec{a},$$ 

can be written as the two differential equations

\begin{align}
    \frac{d\vec{r}}{dt} & = \vec{v}, \\
    \frac{d\vec{v}}{dt} & = \frac{1}{m} \vec{F} .
\end{align}

The equations we used to get an approximate solution for the projectile problem are not accurate enough for our purposes. If you look at the notebook `exercises/02_taylor_series.ipynb`, you'll see the following, more accurate, equations for the position of objects,

\begin{align}
    \vec{r}(t + h) 
& = \color{blue}{\vec{r}(t) + \vec{v}(t) \, h  + \frac{1}{2} \frac{\vec{F}(t)}{m}  \, h^2} + \frac{1}{6} \left( \frac{\vec{F}(t) - \vec{F}(t - h) }{m}\right) \, h^2 + {\cal O}(h^4),
\end{align}

which is accurate to ${\cal O}(h^4)$. The formula for the velocity of an object is

\begin{align}
    \vec{v}(t + h) & = \color{blue}{\vec{v}(t) + \frac{\vec{F}(t)}{m}  \, h}  + \frac{1}{2} \left( \frac{\vec{F}(t) - \vec{F}(t - h) }{m}\right)   \, h^2 + {\cal O}(h^3) ,
\end{align}

which is accurate to ${\cal O}(h^3)$.

Note that at $t = 0$, in general, we do not know the force at time $t = -h$. So a possible simulation strategy is to use the less accurate formulae (in blue) for timestamp $t = +h$ and thereafter use the more accurate ones.

### Import modules 
Make Python modules (that is, collections of programs) available to this notebook.


In [1]:
import os, sys
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib inline

# update fonts
FONTSIZE = 14
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : FONTSIZE}
mp.rc('font', **font)

# use latex if available on system, otherwise set usetex=False
mp.rc('text', usetex=True)

# use JavaScript for rendering animations
mp.rc('animation', html='jshtml')

# set a seed to ensure reproducibility 
# on a given machine
seed = 314159
rnd  = np.random.RandomState(seed)

### Constants

In [2]:
D  = 24*3600.0           # Seconds per day
Y  = 365.25*D            # Seconds per year
Ms = 1.9885e30           # Mass of Sun (kg)
R  = 1.495979e+11        # Astronomical unit (m)
G  = 6.67408e-11         # Gravitational constant (m^3 /kg /s^2)
SIZE = 2*R
h  = D/10                # Time increment in days
hh = h*h

class Bag:               # a very simple class (see PythonMinimum)
    pass
bag    = Bag()

### Exercises

#### Problem 1
Add the masses (in kg in scientific notation) of all the inner planets to the list of Constants above. Use symbols such as `Mm` for Mercury, `Mv` for Venus, etc. 

####  Problem 2. 
Using the data above, and using the matplotlib function `ax.scatter(...)`, plot the predicted positions of the inner planets on 22 September 2024. Use different colored dots for the planets and make the large enough to be clearly visible. 

Assume that all planets lie in the $xy$-plane in the heliocentric coordinate system. (This is approximately true.)

#### Problem 3.
Using the data above compute the position and velocity vectors for the inner planets in SI units (m, kg, s) units on 22 September 2024.
  

#### Problem 4.
  Convert the position and velocities so that they are in astronomical units (au) and au/Earth year.

### Animated Plot of Planetary Motion

In [5]:
class SolarSystem:
    '''
    
    Example
    
    aplot = SolarSystem(nframes)
    aplot.show()
    
    '''
    
    # constructor.
    # this function, which is always called __init__(...), creates and initializes 
    # an object of this class
    
    def __init__(self, data, 
                 nframes, 
                 ftsize=14, 
                 fgsize=(5, 5)):
        '''
        
        data:         results of computations
        nframes:      number of frames
        '''

        # cache inputs
        self.data = data
        self.nframes = nframes

        # set size of figure
        fig = plt.figure(figsize=fgsize)
        self.fig = fig # cache object within AnimPlot
        
        # create area for a single plot 
        nrows, ncols, index = 1, 1, 1
        ax  = plt.subplot(nrows, ncols, index)
        self.ax = ax

        # define graph domain
        ax.set_xlim(xmin, xmax)

        # define graph range   
        ax.set_ylim(ymin, ymax)

        # annotate axes
        ax.set_xlabel('$x$', fontsize=ftsize)
        ax.set_ylabel('$y$', fontsize=ftsize)

        # NB: cache plot
        #self.plot, = ax.plot(x, y, color='blue')
        #ax.grid()

        #fig.tight_layout()
        
        # don't show the above plot. Show only the animated version
        plt.close()
        
        # initialize animated plot
        self.ani = FuncAnimation(fig=fig, 
                                 func=self.__update, 
                                 repeat=False, 
                                 frames=nframes, 
                                 interval=100)
        
    
    # this is the function that actually updates the plot    
    def __update(self, frame):
        
        data, nframes, plot = self.data, self.nframes, self.plot
        #plot.set_data(x, y)

        # must return artists!
        return plot
    
    def show(self):
        plt.show()
        return self.ani
    
    def save(self, filename):
        self.ani.save(filename)